# 0. Setting up bcftools
The package `bcftools` was used to extract relevant information from the `candidate_EOPC_variants.vcf` files by following steps from [this page](https://www.htslib.org/download/). The following are the terminal commands that were performed:
```bash
curl -L -o bcftools-1.22.tar.bz2 https://github.com/samtools/bcftools/releases/download/1.22/bcftools-1.22.tar.bz2
tar xvf bcftools-1.22.tar.bz2
mkdir apps
mkdir apps/bcftools
cd bcftools-1.22
./configure --prefix=/apps/bcftools
make
make install
```
The following command was then added to the .zshrc file:
```bash
export PATH=/apps/bcftools:$PATH 
```
***NOTE:*** `pixi.toml` contains all other packages that were used in this project.


# 1. Generating the non-reference allele count matrix

To extract the genotype matrix (genotype of each individual at each locus) from the VCF file, the following command was first performed in terminal:
```bash
bcftools query -f '%CHROM\t%POS\t%ALT\t%ID\t%TYPE[\t%GT]\n' data/raw/candidate_EOPC_variants.vcf > data/processed/genotype_matrix.tsv
```

**This notebook details the steps taken to convert this genotype matrix into a matrix with the same dimensions representing the number of non-reference alleles for each individual at each locus. This matrix was saved as a TSV file called ***allele_counts.tsv***.**

In [1]:
# import necessary files
import pandas as pd
import numpy as np
from pysam import VariantFile

In [2]:
# import VCF file
vcf_in = VariantFile("data/raw/candidate_EOPC_variants.vcf") 
# store the column names for the genotype matrix
header = ["CHROM", "POS", "ALT","ID", "TYPE"] + list((vcf_in.header.samples))

In [3]:
# import the genotype matrix
genotypes = pd.read_csv("data/processed/genotype_matrix.tsv", sep='\t', header=None)
# add the column names to the genotype matrix
genotypes.columns = header
genotypes

,CHROM,POS,ALT,ID,TYPE,BGC_000034,BGC_000052,BGC_000069,BGC_000071,BGC_000083,...,PCiYP00984,PCiYP00986,PCiYP00988,PCiYP00989,PCiYP00990,PCiYP00991,PCiYP00993,PCiYP00994,PCiYP00995,PCiYP00999
0,chr18,15051,G,chr18_15051_A_G,SNP,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
1,chr18,52006,T,chr18_52006_G_T,SNP,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,chr18,54257,G,chr18_54257_A_G,SNP,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3,chr18,86571,G,chr18_86571_A_G,SNP,0/1,0/0,0/1,0/0,0/0,...,1/1,1/1,0/0,0/1,1/1,0/1,0/1,1/1,0/1,1/1
4,chr18,111667,A,chr18_111667_C_A,SNP,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3359,chr18,80133824,C,chr18_80133824_CGTT_C,INDEL,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3360,chr18,80210807,T,chr18_80210807_A_T,SNP,.,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3361,chr18,80235608,A,chr18_80235608_G_A,SNP,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3362,chr18,80238502,AC,chr18_80238502_A_AC,INDEL,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [4]:
# create a dictionary to store the non-reference allele counts for each combination in the genotype matrix
genotype_to_allele_count = {
    '0/0': 0,
    '0/1': 1,
    '1/0': 1,
    '1/1': 2,
    './.': np.nan,
    '.': 0}

# replace the genotype matrix with the non-reference allele counts
allele_counts = genotypes.replace(genotype_to_allele_count)
allele_counts

/var/folders/0g/f33h1kwn62d9xw1_72cq2zxh0000gn/T/ipykernel_41249/3086601153.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  allele_counts = genotypes.replace(genotype_to_allele_count)


,CHROM,POS,ALT,ID,TYPE,BGC_000034,BGC_000052,BGC_000069,BGC_000071,BGC_000083,...,PCiYP00984,PCiYP00986,PCiYP00988,PCiYP00989,PCiYP00990,PCiYP00991,PCiYP00993,PCiYP00994,PCiYP00995,PCiYP00999
0,chr18,15051,G,chr18_15051_A_G,SNP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,chr18,52006,T,chr18_52006_G_T,SNP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,chr18,54257,G,chr18_54257_A_G,SNP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chr18,86571,G,chr18_86571_A_G,SNP,1.0,0.0,1.0,0.0,0.0,...,2.0,2.0,0.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0
4,chr18,111667,A,chr18_111667_C_A,SNP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3359,chr18,80133824,C,chr18_80133824_CGTT_C,INDEL,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3360,chr18,80210807,T,chr18_80210807_A_T,SNP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3361,chr18,80235608,A,chr18_80235608_G_A,SNP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3362,chr18,80238502,AC,chr18_80238502_A_AC,INDEL,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# save dataframe as tsv file for future access
# allele_counts.to_csv("data/processed/allele_counts.tsv", sep='\t', index=False)